In [1]:
import pandas as pd
from database import DB_CONN
from pymongo import UpdateOne, ASCENDING

In [4]:
df = pd.read_excel('ipo_info.xlsx', header=0, dtype={'code':str})

In [5]:
df.head()

,code,name,issueprice,timeToMarket
0,000001,平安银行,40.0,1989-03-10
1,000002,万科A,1.0,1988-12-28
2,000004,国农科技,1.0,1989-12-23
3,000005,世纪星源,10.0,1990-02-28
4,000006,深振业A,10.0,1991-12-01


In [10]:
df = df.set_index('code')
codes = df.index.tolist()
df.issueprice['000001']
DB_CONN['basic'].find_one({'code':'300257'}, projection={'code':True,
                            'issueprice':True, 'timeToMarket':True,
                            '_id':False})
update_requests = []
for i,code in enumerate(codes):
    try:
        update_requests.append(
            UpdateOne(
            {'code':code},
            {'$set':{'issueprice':df.issueprice[code],
                    'timeToMarket':df.timeToMarket[code]}},
            upsert=True))
    except:
        print('code: %s, has problem' % code)

if len(update_requests) > 0:
    update_result = DB_CONN['basic'].bulk_write(update_requests, ordered=False)
    print('填充字段， 字段名: issueprice，数据集：%s，插入：%4d条，更新：%4d条' %
  ('basic', update_result.upserted_count, update_result.modified_count), flush=True)

# for i,code in enumerate(codes):
#     result = DB_CONN['basic'].find_one({'code':code}, projection={'code':True,
#                             'issueprice':True, 'timeToMarket':True,
#                             '_id':False})
#     print(result)

重新整理数据

1. 复权与不复权数据，并写入到数据库中
2. 基本数据，只要名称; 发行价和发行时间都有了

In [2]:
df = pd.read_excel('data/stock_basic.xlsx', header=0, dtype={'code':str})

In [3]:
df = df.set_index('code')
df.head()

,证券代码,证券名称,首发上市日期,首发价格,戴帽摘帽日期
code,,,,,
000001,000001.SZ,平安银行,1991-04-03,40.0,NaN
000002,000002.SZ,万科A,1991-01-29,1.0,NaN
000004,000004.SZ,国农科技,1991-01-14,1.0,"去ST:2011-06-08,去*ST:2010-05-31,*ST:2009-04-21,..."
000005,000005.SZ,世纪星源,1990-12-10,10.0,"去ST:2008-06-25,去*ST:2004-05-18,*ST:2003-05-12,..."
000006,000006.SZ,深振业A,1992-04-27,10.0,NaN


1. 直接将是否ST存入到本地
2. 实时判断

In [4]:
code = '000004'

In [5]:
st_date_info = df.loc[code, '戴帽摘帽日期']

In [39]:
st_date_info

['去ST:2011-06-08',
 '去*ST:2010-05-31',
 '*ST:2009-04-21',
 '去*ST:2007-05-25',
 '*ST:2006-04-21',
 '去ST:2001-03-06',
 'ST:1999-04-27']

In [40]:
pd.isna(df.loc['000001', '戴帽摘帽日期'])

True

In [6]:
st_state = ['ST', '*ST', '去*ST', '去ST']

In [7]:
st_date_info = st_date_info.split(',')
st_date_info

['去ST:2011-06-08',
 '去*ST:2010-05-31',
 '*ST:2009-04-21',
 '去*ST:2007-05-25',
 '*ST:2006-04-21',
 '去ST:2001-03-06',
 'ST:1999-04-27']

1. 任给一个日期，判断到底是不是st状态;
2. 请把st的状态做成若干个区间, 若最新日期处于st状态,则将区间最后日期设为当前日期(today)
3. 去ST代码解除ST状态,ST代表成为ST状态,所以我们只需要找到里面非\*状态的时间点

In [8]:
import datetime

In [9]:
today = str(datetime.date.today())

In [10]:
import numpy as np

In [53]:
np.random.choice(QA.trade_date_sse[5881:-44])

'2015-09-25'

In [54]:
date = np.random.choice(QA.trade_date_sse[:-44])

```python

if it == '去ST':
    这个日期之后数据都是解除ST的状态
    往前找到'ST'
    next one

elif it == 'ST'


```

In [56]:
# if i.split(':')[0] == st_state[0]:
st_date_info[0].split(':')[0]

'去ST'

In [57]:
date < st_date_info[0].split(':')[1]

True

In [23]:
find_tag = ''
st_interval_list = []
for i in st_date_info:
    key = i.split(':')[0]
    value = datetime.datetime.strptime(i.split(':')[1], '%Y-%m-%d')
    if key == '去ST':
        if find_tag == 'ST':
            print('something is wrong, code is value is ')
        else:
            temp_st_end = (value - datetime.timedelta(1)).strftime('%Y-%m-%d')
            find_tag = 'ST'
    
    elif key == 'ST':
        temp_st_start = value.strftime('%Y-%m-%d')
        
        if find_tag == 'ST':
            st_interval_list.append([temp_st_start, temp_st_end])
            find_tag = '去ST'
            
#             if temp_st_end is not None:
#                 st_interval_list.append([temp_st_start, temp_st_end])
#                 temp_st_end = None
#                 find_tag = '去ST'
#             else:
#                 print('something is wrong, code is value is ')
        
        elif find_tag == '':
            st_interval_list.append([temp_st_start, today])
            find_tag = '去ST'

something is wrong, code is value is 


In [24]:
st_interval_list

[['1999-04-27', '2011-06-07']]

In [31]:
st_interval_list = []
temp_st_end = None
for count, i in enumerate(st_date_info):
    key = i.split(':')[0]
    value = datetime.datetime.strptime(i.split(':')[1], '%Y-%m-%d')
    
    if key == '去ST' and count==0:
        temp_st_end = (value - datetime.timedelta(1)).strftime('%Y-%m-%d')
    
    elif key == '去ST' and count >0:
        temp_st_start = datetime.datetime.strptime(st_date_info[count-1].split(':')[1], 
                                   '%Y-%m-%d').strftime('%Y-%m-%d')
        st_interval_list.append([temp_st_start, temp_st_end])
        temp_st_end = (value - datetime.timedelta(1)).strftime('%Y-%m-%d')
    elif count == (len(st_date_info)-1):
        if temp_st_end is not None:
            st_interval_list.append([value.strftime('%Y-%m-%d'), temp_st_end])
        else:
            st_interval_list.append([value.strftime('%Y-%m-%d'), today])
        

In [32]:
st_interval_list

[['2006-04-21', '2011-06-07'], ['1999-04-27', '2001-03-05']]

In [ ]:
pd.concat([pd.date_range(st_interval_list[0][0], st_interval_list[0][1]),
pd.date_range(st_interval_list[1][0], st_interval_list[1][1])])

In [38]:
pd.date_range(st_interval_list[0][0], st_interval_list[0][1])

DatetimeIndex(['2006-04-21', '2006-04-22', '2006-04-23', '2006-04-24',
               '2006-04-25', '2006-04-26', '2006-04-27', '2006-04-28',
               '2006-04-29', '2006-04-30',
               ...
               '2011-05-29', '2011-05-30', '2011-05-31', '2011-06-01',
               '2011-06-02', '2011-06-03', '2011-06-04', '2011-06-05',
               '2011-06-06', '2011-06-07'],
              dtype='datetime64[ns]', length=1874, freq='D')

In [21]:
(datetime.datetime.strptime('2011-06-08', '%Y-%m-%d') - \
 datetime.timedelta(1)).strftime('%Y-%m-%d')

'2011-06-07'